# スーパーバイザー型マルチエージェントシステムの構築 - MLflow & LangGraph Tutorial

## 概要
このノートブックでは、複数の専門エージェントが協調して複雑なタスクを遂行する「スーパーバイザー型マルチエージェントシステム」を構築します。技術レポート作成という実践的なユースケースを通じて、エージェント設計、協調パターン、評価手法を学びます。

### 学習内容
1. マルチエージェントシステムの設計パターン（スーパーバイザー型）
2. 専門化されたエージェントの実装（リサーチ、構成、執筆、レビュー）
3. MLflow ResponseAgentによる標準インターフェース化
4. 多様な評価手法（Safety、Guidelines、カスタムスコアラー、Agent-as-a-Judge）
5. トレースベースの品質分析

### スーパーバイザー型アーキテクチャとは？

**従来の単一エージェント**
- 1つのLLMがすべてのタスクを担当
- 複雑なタスクでは品質が低下

**スーパーバイザー型マルチエージェント**
- 各エージェントが専門領域に特化
- スーパーバイザーが全体を調整し、適切なエージェントに仕事を割り当て
- 段階的な処理により高品質な出力を実現

### 本ノートブックのエージェント構成

1. **リサーチエージェント**: テーマの調査とポイント整理
2. **構成エージェント**: レポート構成と見出しの決定
3. **ライティングエージェント**: 本文の執筆
4. **レビューエージェント**: 品質チェックと修正
5. **スーパーバイザー**: 処理フローの制御と次のエージェントの決定

## ステップ1: 環境セットアップ

### 必要なライブラリ
- `mlflow`: 実験管理、モデル記録、トレーシング、評価
- `langchain[openai]`: LangChainとOpenAI連携
- `langgraph`: マルチエージェントワークフローの構築
- `litellm`: 複数のLLMプロバイダーへの統一インターフェース

In [ ]:
%pip install mlflow langchain[openai] langgraph litellm

## ステップ2: 認証情報の設定

OpenAI APIを使用するため、APIキーを環境変数に設定します。

**重要**: 
- `YOUR_API_KEY`を実際のAPIキーに置き換えてください
- 本番環境では、シークレット管理サービスを使用することを推奨します

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

## ステップ3: マルチエージェントシステムの実装

### アーキテクチャの詳細

#### 共有状態（AgentState）
すべてのエージェントが参照・更新する共通のデータ構造です。
- `topic`: レポートのテーマ
- `research_notes`: リサーチエージェントの出力
- `outline`: 構成エージェントの出力
- `draft`: ライティングエージェントの出力
- `review_comments`: レビューエージェントのコメント
- `final_report`: 最終的なレポート
- `next_agent`: スーパーバイザーが決定する次の担当エージェント

#### 処理フロー
```
START → Supervisor → Research Agent → Supervisor
                   → Outline Agent → Supervisor
                   → Writer Agent → Supervisor
                   → Review Agent → Supervisor → END
```

### エージェント設計のポイント

**専門化**
- 各エージェントは1つの明確な役割のみを担当
- プロンプトは役割に特化した指示を含む

**状態の受け渡し**
- 前のエージェントの出力を次のエージェントの入力として利用
- スーパーバイザーが状態を確認し、次のステップを判断

**トレーサビリティ**
- MLflow Tracingで各エージェントの処理を記録
- デバッグや品質分析が容易

In [ ]:
%%writefile ./multi_agent_report_app.py
"""
スーパーバイザー型マルチエージェントによる技術レポート作成アプリケーションです。

エージェントの役割:
  - リサーチエージェント: テーマについて調査し、ポイントを箇条書きで整理
  - 構成エージェント: 見出し構成と各見出しの要点を決定
  - ライティングエージェント: 構成に沿って本文を執筆
  - レビューエージェント: レポート案をチェックし、必要なら修正を提案

スーパーバイザー:
  - 全体を調整し、各エージェントに順番に仕事を振り分ける

MLflow ResponseAgent:
  - システム全体をResponseAgentでラッピングし、標準的なインターフェースで公開します。
"""

from __future__ import annotations

from typing import TypedDict, Literal, Annotated
import functools

import mlflow
from mlflow.entities import SpanType

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent
from langgraph.graph.message import add_messages


# ==========
# 事前準備
# ==========

# LLM（全エージェント共通）
# temperature=0.2で適度な創造性を保ちつつ、一貫性のある出力を実現
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

# LangChain/MLflowの自動ロギングを有効化
# LLM呼び出しやエージェント実行を自動的にMLflowに記録
mlflow.langchain.autolog()


# ==========
# 状態の定義
# ==========

class AgentState(TypedDict):
    """
    エージェント間で受け渡す情報をまとめた共有状態です。
    
    この状態はワークフロー全体を通じて維持され、
    各エージェントが必要な情報を読み取り、自分の出力を書き込みます。
    """
    topic: str              # レポートのテーマ
    research_notes: str     # リサーチ結果（箇条書き）
    outline: str            # レポート構成（見出しと要点）
    draft: str              # レポート本文（初稿）
    review_comments: str    # レビューコメント
    final_report: str       # 最終レポート（修正済み）
    next_agent: str         # スーパーバイザーが決定する次のエージェント名


# ==========
# 各エージェントノードの実装
# ==========

def create_agent_node(agent_name: str, system_prompt: str):
    """
    エージェントノードを作成するファクトリー関数です。
    
    このパターンにより、コードの重複を避け、
    各エージェントの設定を一元管理できます。
    
    Args:
        agent_name: エージェントの識別名
        system_prompt: エージェントの役割と指示を定義するプロンプト
    
    Returns:
        エージェントノード関数
    """
    def agent_node(state: AgentState) -> AgentState:
        """エージェントの処理を実行し、状態を更新します。"""
        # 状態から必要な情報を取り出す
        topic = state.get("topic", "")
        research = state.get("research_notes", "")
        outline = state.get("outline", "")
        draft = state.get("draft", "")

        # エージェントごとに異なる入力を構築
        # 各エージェントは前段階の出力を入力として受け取る
        if agent_name == "research_agent":
            user_content = f"テーマ: {topic}"
        elif agent_name == "outline_agent":
            user_content = f"テーマ: {topic}\n\nリサーチ結果:\n{research}"
        elif agent_name == "writer_agent":
            user_content = f"テーマ: {topic}\n\n構成案:\n{outline}"
        elif agent_name == "review_agent":
            user_content = f"レポートドラフト:\n{draft}"
        else:
            user_content = ""

        # LLMを呼び出し
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_content),
        ]
        response = llm.invoke(messages)
        result = response.content

        # 結果を状態に保存
        # 各エージェントは自分の担当フィールドのみを更新
        if agent_name == "research_agent":
            state["research_notes"] = result
        elif agent_name == "outline_agent":
            state["outline"] = result
        elif agent_name == "writer_agent":
            state["draft"] = result
        elif agent_name == "review_agent":
            # レビュー結果を解析して、コメントと最終レポートに分割
            if "【修正後レポート案】" in result:
                comments, final = result.split("【修正後レポート案】", maxsplit=1)
                state["review_comments"] = comments.strip()
                state["final_report"] = final.strip()
            else:
                state["review_comments"] = result
                state["final_report"] = draft  # 修正不要の場合は原稿をそのまま使用

        # トレースにプレビューを記録（後で分析しやすくするため）
        mlflow.update_current_trace(tags={
            f"{agent_name}_preview": result[:100],  # 最初の100文字のみ
        })

        return state

    return agent_node


# 4つの専門エージェントを作成
# 各エージェントは明確な役割と具体的な出力形式を持つ

research_node = create_agent_node(
    "research_agent",
    "あなたは技術リサーチ担当です。テーマについて重要なポイントを箇条書きで5〜7個挙げてください。"
)

outline_node = create_agent_node(
    "outline_agent",
    "あなたは技術レポートの構成を考える担当です。リサーチメモをもとに、見出し構成（3〜5個）と各見出しの要点を番号付きで出力してください。"
)

writer_node = create_agent_node(
    "writer_agent",
    "あなたは技術レポートの執筆担当です。構成案に従って、各見出しごとに2〜4文程度で本文を書いてください。専門用語は平易な言葉で説明してください。"
)

review_node = create_agent_node(
    "review_agent",
    "あなたは技術レポートのレビュー担当です。技術的な正確さ、構成のわかりやすさ、文体をチェックし、【コメント】と【修正後レポート案】の形式で出力してください。"
)


# ==========
# スーパーバイザーノード
# ==========

def supervisor_node(state: AgentState) -> AgentState:
    """
    【スーパーバイザー: ワークフローの制御塔】
    
    現在の状態を確認し、次にどのエージェントを呼ぶかを決定します。
    
    判断ロジック（順次処理）:
    1. リサーチ結果がない → research_agent
    2. 構成案がない → outline_agent
    3. 本文がない → writer_agent
    4. 最終レポートがない → review_agent
    5. すべて完了 → FINISH
    """
    # 状態を見て、次に呼ぶべきエージェントを判断
    if not state.get("research_notes"):
        next_agent = "research_agent"
    elif not state.get("outline"):
        next_agent = "outline_agent"
    elif not state.get("draft"):
        next_agent = "writer_agent"
    elif not state.get("final_report"):
        next_agent = "review_agent"
    else:
        next_agent = "FINISH"

    state["next_agent"] = next_agent

    # トレースに判断結果を記録
    mlflow.update_current_trace(tags={
        "next_agent": next_agent,
    })

    return state


# ==========
# グラフの構築
# ==========

def build_graph():
    """
    スーパーバイザー型のマルチエージェントグラフを構築します。
    
    グラフ構造:
    - 中央にスーパーバイザーを配置
    - 各エージェントはスーパーバイザーから呼び出され、完了後にスーパーバイザーに戻る
    - スーパーバイザーが次のエージェントを決定（条件分岐）
    """
    workflow = StateGraph(AgentState)

    # 5つのノードを追加（スーパーバイザー + 4つの専門エージェント）
    workflow.add_node("supervisor", supervisor_node)
    workflow.add_node("research_agent", research_node)
    workflow.add_node("outline_agent", outline_node)
    workflow.add_node("writer_agent", writer_node)
    workflow.add_node("review_agent", review_node)

    # 開始点: まずスーパーバイザーから開始
    workflow.add_edge(START, "supervisor")

    # スーパーバイザーの判断に基づいて分岐
    def route_supervisor(state: AgentState) -> Literal["research_agent", "outline_agent", "writer_agent", "review_agent", "__end__"]:
        """スーパーバイザーの決定に基づいて次のノードを返す"""
        next_agent = state.get("next_agent", "FINISH")
        if next_agent == "FINISH":
            return END
        return next_agent

    workflow.add_conditional_edges(
        "supervisor",
        route_supervisor,
        {
            "research_agent": "research_agent",
            "outline_agent": "outline_agent",
            "writer_agent": "writer_agent",
            "review_agent": "review_agent",
            END: END,
        },
    )

    # 各エージェント実行後は、必ずスーパーバイザーに戻る（固定エッジ）
    # これにより、スーパーバイザーが次のステップを制御できる
    for agent in ["research_agent", "outline_agent", "writer_agent", "review_agent"]:
        workflow.add_edge(agent, "supervisor")

    return workflow.compile()


# グラフを構築
graph = build_graph()


# ==========
# ResponseAgentでラッピング
# ==========

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from typing import Generator

class MultiAgentResponsesAgent(ResponsesAgent):
    """
    マルチエージェントシステムをResponsesAgentでラッピングします。
    
    ResponsesAgentとは？
    - MLflowが提供する標準的なエージェントインターフェース
    - OpenAI互換のAPI形式でサービング可能
    - ストリーミングと非ストリーミングの両方に対応
    
    メリット:
    - 既存のチャットアプリケーションとの統合が容易
    - REST API、Python、CLI等、複数の方法で呼び出し可能
    - MLflowの評価・モニタリング機能と統合
    """

    def __init__(self, graph):
        self.graph = graph

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """
        非ストリーミング版の予測メソッド。
        ResponsesAgentRequestを受け取り、マルチエージェントを実行します。
        
        Args:
            request: OpenAI互換の入力リクエスト
        
        Returns:
            ResponsesAgentResponse: 最終レポートを含むレスポンス
        """
        # ストリーミング版を実行して、完了イベントだけを集める
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]

        return ResponsesAgentResponse(
            output=outputs,
            custom_outputs=request.custom_inputs
        )

    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        ストリーミング版の予測メソッド。
        各エージェントの出力をリアルタイムで返すことも可能ですが、
        ここでは最終結果のみを返す実装としています。
        
        Args:
            request: OpenAI互換の入力リクエスト
        
        Yields:
            ResponsesAgentStreamEvent: ストリーミングイベント
        """
        # RequestをChatCompletions形式に変換
        messages = to_chat_completions_input([i.model_dump() for i in request.input])

        # ユーザーの質問からトピックを抽出
        topic = messages[-1]["content"] if messages else ""

        # グラフに渡す初期状態を作成
        initial_state = AgentState(
            topic=topic,
            research_notes="",
            outline="",
            draft="",
            review_comments="",
            final_report="",
            next_agent="",
        )

        # グラフを実行（すべてのエージェントが順次実行される）
        final_state = self.graph.invoke(initial_state)

        # 最終レポートを出力として返す
        final_report = final_state.get("final_report", "")

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item(
                text=final_report,
                id="final_report",
            )
        )


# エージェントをインスタンス化してMLflowに登録
agent = MultiAgentResponsesAgent(graph)
mlflow.models.set_model(agent)

## ステップ4: マルチエージェントシステムの実行とテスト

構築したマルチエージェントシステムに実際にレポート作成を依頼します。

### ResponsesAgentRequestの形式
OpenAI ChatCompletions APIと互換性のある形式です。
- `type`: メッセージタイプ（"message"）
- `role`: 役割（"user", "assistant", "system"）
- `content`: メッセージ内容（テキストや画像など）

In [ ]:
import mlflow
from multi_agent_report_app import agent

# レポート作成のリクエスト
question = "RAGに関する技術レポートを書いてください"

# ResponsesAgentRequestの形式
# OpenAI互換のメッセージ形式
input_data = {
    "input": [
        {
            "type": "message",
            "role": "user",
            "content": [{"type": "input_text", "text": question}]
        }
    ]
}

# エージェントを実行
# 内部で research → outline → writer → review と順次実行される
response = agent.predict(input_data)

# 最終レポートを取得
final_report = response.output[0].content[0]["text"]

print("質問：", question)
print("\n最終レポート：")
print(final_report)

## ステップ5: ワークフローの可視化

マルチエージェントシステムの構造を可視化します。

### 可視化で確認できること
- スーパーバイザーと各エージェントの関係
- 条件分岐の構造
- 処理の循環パターン（エージェント → スーパーバイザー → 次のエージェント）

In [ ]:
from IPython.display import Image, display
from multi_agent_report_app import graph

try:
    # エージェントのグラフ構造を可視化（Mermaid形式）
    graph_image = graph.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    print("✓ ワークフローの図を表示しました")
except Exception as e:
    print(f"図の表示に失敗しました: {e}")
    print("（この機能は環境によっては動作しない場合があります）")

## ステップ6: MLflowへのモデル記録

### PyFuncフレーバー
ResponsesAgentはMLflowのPyFuncフレーバーとして記録できます。

### メタデータの記録
モデルと一緒に以下の情報も記録します：
- **パラメータ**: エージェント数、スーパーバイザータイプ等
- **タグ**: アーキテクチャタイプ、バージョン等
- **入力例**: スキーマ推論とテスト用

In [ ]:
import mlflow

# MLflowの設定（ローカルTrackingサーバーを想定）
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("agentic_rag_example")

# LangGraphで構築したマルチエージェントシステムをMLflowに記録
with mlflow.start_run(run_name="multi-agent-report-v1") as run:
    # pyfuncフレーバーとして記録
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model="./multi_agent_report_app.py",
        input_example=input_data,
    )

    # パラメータとタグを記録（後で検索・比較しやすくする）
    mlflow.log_param("agent_count", 4)
    mlflow.log_param("supervisor_type", "sequential")
    mlflow.set_tag("architecture", "supervisor")

print("モデルURI:", model_info.model_uri)

## ステップ7: モデルのロードと推論テスト

記録したモデルを読み込み、推論が正しく動作することを確認します。

In [ ]:
# 記録したモデルを読み込む
loaded = mlflow.pyfunc.load_model(model_info.model_uri)

# テスト推論を実行
response = loaded.predict(input_data)

# 結果を取得
final_report = response["output"][0]["content"][0]["text"]
print("推論結果:")
print(final_report)

## ステップ8: MLflow Evaluation - Safety評価

### Safety評価とは？
LLMの出力が安全で適切かを評価する指標です。

### 評価観点
- 有害なコンテンツの有無
- 偏見や差別的表現
- 不適切な言及

### 使用シーン
本番環境にデプロイする前の安全性チェックとして活用します。

In [ ]:
"""
マルチエージェントシステムをMLflow Evaluationで評価する例です。
まずはSafety（安全性）評価から始めます。
"""
from __future__ import annotations

import mlflow
from mlflow.genai.scorers import Safety

# 1. 評価データセットを用意
dataset = [
    {
        "inputs": {
            "question": "RAGに関するレポートを作成してください",
        },
    },
]

# 2. 予測関数を定義
def predict_wrapper(question: str) -> str:
    """
    MLflow Evaluationから呼び出される予測関数です。

    Args:
        question: レポートのテーマ

    Returns:
        生成されたレポート
    """
    # ResponsesAgentRequestの形式
    input_data = {
        "input": [
            {
                "type": "message",
                "role": "user",
                "content": [{"type": "input_text", "text": question}]
            }
        ]
    }

    response = loaded.predict(input_data)
    final_report = response["output"][0]["content"][0]["text"]
    return final_report

In [ ]:
# 3. Safety評価の実行
with mlflow.start_run():
    results = mlflow.genai.evaluate(
        data=dataset,
        predict_fn=predict_wrapper,
        scorers=[
            Safety(model="openai:/gpt-3.5-turbo"),
        ],
    )

print("安全性評価が完了しました。MLflow UIで結果を確認してください。")

## ステップ9: MLflow Evaluation - Guidelinesベースの評価

### Guidelinesスコアラーとは？
自然言語で記述された評価基準に基づいて、LLMが出力を評価する仕組みです。

### メリット
- **カスタマイズ性**: ビジネス要件に合わせた独自の評価基準を設定可能
- **可読性**: 評価基準が自然言語なので、非技術者でも理解しやすい
- **柔軟性**: コードを書かずに評価基準を変更できる

### 評価基準の例
1. **トーン**: 丁寧でプロフェッショナルな文体か
2. **理解しやすさ**: 明確で簡潔な説明か
3. **禁止トピック**: 特定の内容が含まれていないか

In [ ]:
from mlflow.genai import scorers

"""
ガイドラインベースのLLMスコアラー

ガイドラインは、合格/不合格条件として定義された自然言語基準を定義することで、
評価を迅速かつ容易にカスタマイズできるように設計された強力なスコアラークラスです。
ルール、スタイルガイド、情報の包含/除外への準拠チェックに最適です。

ガイドラインには、ビジネス関係者への説明が容易であるという明確な利点があります
（「アプリがこのルールセットを満たしているかどうかを評価しています」）。
そのため、多くの場合、ドメインエキスパートが直接記述できます。
"""

# 自然言語で評価基準を定義
tone = "回答は終始、丁寧でプロフェッショナルさを保たねばならない。"

easy_to_understand = """回答は明確かつ簡潔な言葉を用い、論理的に構成されなければなりません。
専門用語の使用は避け、使用する場合は説明を加える必要があります。"""

banned_topics = "価格に関する具体的な数値が記載されていないこと"

# Guidelinesスコアラーを作成
tone_scorer = scorers.Guidelines(
    name="tone", 
    model="openai:/gpt-3.5-turbo",
    guidelines=tone
)

easy_to_understand_scorer = scorers.Guidelines(
    name="easy_to_understand", 
    model="openai:/gpt-3.5-turbo",
    guidelines=easy_to_understand
)

banned_topics_scorer = scorers.Guidelines(
    name="banned_topics", 
    model="openai:/gpt-3.5-turbo",
    guidelines=banned_topics
)

### トレースベースの評価

MLflowは、過去の実行トレースを直接評価できます。
これにより、本番環境での実際の出力を事後的に評価することが可能です。

In [ ]:
# 評価の実行例
# 直近のトレースを取得
traces = mlflow.search_traces(
    max_results=1,
)

if traces.empty:
    print("評価対象のトレースが見つかりません。")
    raise SystemExit(1)

# ガイドラインベースの評価を実行
results = mlflow.genai.evaluate(
    data=traces,
    scorers=[tone_scorer, easy_to_understand_scorer, banned_topics_scorer],
)

print("自然言語ベースの評価が完了しました。")

## ステップ10: カスタムスコアラー - エージェント網羅性評価

### カスタムスコアラーとは？
独自のロジックで評価を行うスコアラーです。

### このスコアラーの目的
マルチエージェントシステムで、期待されるすべてのエージェントが
実際に呼び出されたかを確認します。

### 実装のポイント
- `@scorer`デコレータで関数をスコアラーとして登録
- `Trace`オブジェクトからSpanを検索
- `Feedback`オブジェクトでスコアと理由を返す

In [ ]:
"""
エージェント呼び出しの網羅性を評価するコードベースのカスタムScorerです。

・トレースからSpanType.AGENTのスパンを抽出
・実際に呼ばれたエージェント名のリストを取得
・期待されるエージェントがすべて呼ばれたかを確認
"""
import mlflow
from mlflow.entities import Feedback, Trace, SpanType
from mlflow.genai import scorer
import pandas as pd

@scorer
def agent_coverage(trace: Trace, expectations: dict) -> Feedback:
    """
    想定通りのエージェントが呼ばれているかを評価します。

    評価基準:
    - 期待されるエージェントがすべて呼ばれた場合: スコア1.0
    - 一部のエージェントが欠けている場合: 呼ばれた割合をスコアとする

    Args:
        trace: 評価対象のトレース
        expectations: 期待される動作を定義した辞書

    Returns:
        Feedback: スコアと理由を含む評価結果
    """
    # トレースからエージェントスパンを検索
    agent_spans = trace.search_spans(span_type=SpanType.AGENT)

    # 実際に呼び出されたエージェント名を抽出
    invoked_agents = [span.name for span in agent_spans]

    # 期待されるエージェントのリスト
    expected_agents = expectations.get("expected_agents", [])
    if not expected_agents:
        return Feedback(
            value=1.0,
            rationale="期待値が指定されていないため、評価をスキップしました。"
        )

    # 網羅性を計算（期待されるエージェントのうち何%が呼ばれたか）
    invoked_set = set(invoked_agents)
    expected_set = set(expected_agents)
    covered = invoked_set & expected_set
    coverage_ratio = len(covered) / len(expected_set) if expected_set else 0

    # 詳細な理由を生成
    if coverage_ratio == 1.0:
        rationale = f"期待通り、すべてのエージェント {expected_agents} が呼び出されました。"
    else:
        missing = expected_set - invoked_set
        extra = invoked_set - expected_set
        rationale = (
            f"エージェント網羅率: {coverage_ratio:.0%}\n"
            f"期待: {sorted(expected_agents)}\n"
            f"実際: {sorted(invoked_agents)}\n"
        )
        if missing:
            rationale += f"未呼び出し: {sorted(missing)}\n"
        if extra:
            rationale += f"予期しない呼び出し: {sorted(extra)}"

    return Feedback(
        value=coverage_ratio,
        rationale=rationale.strip()
    )

# トレースから評価データを取得
traces = mlflow.search_traces(
    max_results=1,
)

if traces.empty:
    print("評価対象のトレースが見つかりません。")
    raise SystemExit(1)

# 期待されるエージェントのリストを追加
traces["expectations"] = [{
    "expected_agents": [
        "research_agent",
        "outline_agent",
        "writer_agent",
        "review_agent"
    ]
}] * len(traces)

# 評価を実行
results = mlflow.genai.evaluate(
    data=traces,
    scorers=[agent_coverage],
)

print("エージェント網羅性の評価が完了しました。MLflow UIで結果を確認してください。")

## ステップ11: Agent-as-a-Judge評価

### Agent-as-a-Judgeとは？
LLMエージェントが評価者となり、複雑な評価基準に基づいて
他のLLMの出力を評価する手法です。

### 他の評価手法との違い

| 手法 | 評価方法 | 適用範囲 |
|------|----------|----------|
| コードベース | ルールベースチェック | 限定的 |
| Guidelinesベース | セマンティックなチェック | 中程度 |
| Agent-as-a-Judge | 複雑な推論と総合判断 | 広範囲 |

### このスコアラーの評価観点
**エージェント間の協調性**を多角的に評価：
1. リサーチ結果が構成案に適切に反映されているか
2. 構成案が本文に適切に反映されているか
3. レビューコメントが本文の内容と整合しているか
4. 無駄な差し戻しや重複作業が発生していないか

In [ ]:
"""
Agent-based Scorer (aka. Agent-as-a-Judge)を使った評価の例です。

MLflowでは、評価基準を自然言語で記述することで、
エージェントが自動的にその基準に基づいて評価を行います。
"""
import mlflow
from mlflow.genai.judges import make_judge
from typing import Literal

# 自然言語で評価基準を定義
AGENT_COORDINATION_GUIDELINE = """
あなたは、マルチエージェントシステムの「エージェント間の協調性」を評価する役割です。
以下の観点で {{ trace }} を評価し、スコア（1〜5の整数）と理由を返してください。

評価観点:
1. リサーチ結果が構成案に適切に反映されているか
2. 構成案が本文に適切に反映されているか
3. レビューコメントが本文の内容と整合しているか
4. 無意味な差し戻しや重複作業が発生していないか

スコアの基準:
- 5: すべての観点で優れている
- 4: ほとんどの観点で良好だが、軽微な改善点がある
- 3: いくつかの観点で問題があるが、全体としては機能している
- 2: 複数の観点で明確な問題がある
- 1: エージェント間の連携が機能していない
"""

# Agent-as-a-Judgeスコアラーを作成
agent_coordination_scorer = make_judge(
    name="agent_coordination",
    instructions=AGENT_COORDINATION_GUIDELINE,
    feedback_value_type=Literal["5", "4", "3", "2", "1"],
    # トレース全体を分析するため、モデルを指定
    model="openai:/gpt-4o-mini",
)

# トレースから評価データを取得
traces = mlflow.search_traces(
    max_results=1,
)

if traces.empty:
    print("評価対象のトレースが見つかりません。")
    raise SystemExit(1)

# Agent-as-a-Judge評価を実行
results = mlflow.genai.evaluate(
    data=traces,
    scorers=[agent_coordination_scorer],
)

print("エージェント協調性の評価が完了しました。MLflow UIで結果を確認してください。")

## まとめ

### このノートブックで学んだこと

#### アーキテクチャ設計
1. **スーパーバイザー型マルチエージェント**: 中央制御による効率的なタスク分割
2. **エージェントの専門化**: 各エージェントが明確な役割を持つ設計
3. **状態管理**: AgentStateによる情報の受け渡し
4. **ResponsesAgent**: 標準インターフェースでのラッピング

#### 評価手法の多様性
1. **Safety評価**: 安全性の自動チェック
2. **Guidelinesベース評価**: ビジネスルールの自然言語記述
3. **カスタムスコアラー**: 独自ロジックによる評価（エージェント網羅性）
4. **Agent-as-a-Judge**: 複雑な総合評価（協調性）

#### MLflowの活用
1. **Tracing**: 全エージェントの処理を詳細に記録
2. **Model Registry**: バージョン管理とデプロイ準備
3. **Evaluation**: 多角的な品質評価
4. **タグとパラメータ**: 実験の整理と検索

### スーパーバイザー型の利点

- **品質向上**: 各段階で専門的な処理が可能
- **デバッグ容易性**: 問題のあるエージェントを特定しやすい
- **拡張性**: 新しいエージェントの追加が容易
- **制御性**: スーパーバイザーで処理フローを柔軟に制御

### 次のステップ

#### 機能拡張
- 並列処理: 独立したエージェントを同時実行
- 動的ルーティング: LLMによる次エージェントの判断
- エラーハンドリング: リトライ機能の追加
- メモリ機能: 過去の実行結果を参照

#### 評価の強化
- より大規模な評価データセット
- 人間による評価との比較
- A/Bテストによるプロンプト最適化
- 継続的評価パイプラインの構築

#### 本番環境へのデプロイ
- REST APIとしてのサービング
- バッチ処理パイプラインの構築
- モニタリングとアラート設定
- コスト最適化（モデル選択、キャッシング）

### 参考リソース

- [LangGraph Multi-Agent Systems](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)
- [MLflow Tracing](https://mlflow.org/docs/latest/llms/tracing/index.html)
- [MLflow Evaluation for LLMs](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html)
- [MLflow ResponsesAgent](https://mlflow.org/docs/latest/llms/deployments/index.html)
- [Agent-as-a-Judge Pattern](https://arxiv.org/abs/2410.10934)

---